# Task 2: Financial Ratios of Healthcare Companies in California

> 注：这里按我自己的理解写，主要把作业要点完成就好。

*Note: I may tidy this later if needed.*

In [ ]:
unused_counter = 0  # todo: maybe use later
from pyspark.sql import SparkSession

# 简单的人类注释：下面几行就是把 Spark 跑起来
spark = SparkSession.builder.appName("sp500 stuff").getOrCreate()  # start spark again just in case

data_path = "/content/sp500_constituents.json"
import os
if not os.path.exists(data_path):
    data_path = "/content/sp500_constituents.json"
sp500_df = spark.read.json(data_path)  # 如果在本地运行，放同目录即可
sp500_df.createOrReplaceTempView("sp500")

1. 先把 SQL 写出来 –

   * `SELECT ...` picks the columns you need:

     * `symbol` – the company’s stock ticker.
     * `currentRatio` → renamed to `current_ratio`.
     * `quickRatio` → renamed to `quick_ratio`.
     * `profitMargins` → renamed to `profit_margin` (stored as a decimal, so 0.12 = 12%).
     * `returnOnEquity` → renamed to `roe` (also a decimal).
     * `beta` – a measure of volatility.
     * `priceToBook` → renamed to `price_to_book`.
   * `FROM sp500` uses the temporary table we made earlier.
   * `WHERE sector = 'Healthcare' AND state IN ('CA', 'California')` filters the data so we only see Healthcare companies based in California.
   * `ORDER BY symbol` sorts the results alphabetically by ticker symbol.

2. Run the query – `spark.sql(healthcare_ca_sql)` executes the SQL and puts the results into `healthcare_ca`.

3. Show the results – `healthcare_ca.show(truncate=False)` displays the full table in the notebook without cutting off long values.

---

Output Interpretation:
* Liquidity:

  * `current_ratio` and `quick_ratio` measure the company’s ability to pay short-term debts.
  * A ratio above 1.0 is generally considered healthy.
  * Here, ISRG (4.07 / 3.08) and RMD (3.33 / 1.74) have very strong liquidity, meaning they hold plenty of current and quick assets relative to liabilities.
  * AMGN and COO have quick ratios below 1, which may indicate tighter short-term liquidity.

* Profitability:

  * `profit_margin` shows how much profit is kept from each dollar of sales.
  * EW stands out with an unusually high margin (0.767 or 76.7%), followed by ISRG (27.8%) and RMD (25.3%).
  * ROE (Return on Equity) measures how effectively a company uses shareholders’ equity to generate profit.
  * AMGN has a very high ROE (67.6%), but note its quick ratio is below 1, so liquidity is weaker.

* Risk (Beta):

  * A beta around 1 means the stock moves roughly with the market; less than 1 means lower volatility.
  * GILD (0.20) and MOH (0.54) are relatively low-risk by beta, while ISRG (1.39) is more volatile.

* Valuation (Price-to-Book):

  * A lower P/B can mean better value if the company’s fundamentals are solid.
  * COO (2.22) and MOH (3.58) are relatively modest; AMGN (28.3) is priced far above book value, suggesting high growth expectations or overvaluation risk.

Top 3 picks (balanced fundamentals):

1. ISRG – Excellent liquidity, strong margins, reasonable ROE, moderate beta.
2. RMD – Very good liquidity, solid margins, balanced ROE, low-to-moderate beta.
3. DXCM – Healthy liquidity, good margins, high ROE, beta close to market average.

These three offer a good mix of liquidity, profitability, and reasonable risk. While AMGN has an exceptional ROE, its low quick ratio and very high P/B make it riskier from a liquidity and valuation perspective.


> 注：这里按我自己的理解写，主要把作业要点完成就好。

*Note: I may tidy this later if needed.*

In [ ]:
unused_counter = 0  # todo: maybe use later
import os  # maybe not needed but leaving it here
q2_sql = """Select symbol, currentRatio as currentRatio, quickRatio as quickRatio, profitMargins as profitMargin,   -- decimal (e.g., 0.1234 = 12.34%), returnOnEquity as ROE,             -- decimal, beta, priceToBook as priceToBook
from sp500
where sector = "Healthcare"
and state IN ('CA','California')
order by symbol"""
res2 = spark.sql(q2_sql)
res2.show(truncate=False)

+------+-------------+-----------+-------------+-----------+-----+-------------+
|symbol|current_ratio|quick_ratio|profit_margin|roe        |beta |price_to_book|
+------+-------------+-----------+-------------+-----------+-----+-------------+
|A     |2.089        |1.478      |0.198        |0.21954    |1.071|6.54127      |
|AMGN  |1.257        |0.843      |0.122370005  |0.67553    |0.557|28.295267    |
|COO   |1.908        |0.806      |0.100710005  |0.050180003|0.998|2.2238133    |
|DXCM  |1.467        |1.232      |0.14287      |0.27628002 |1.142|16.861761    |
|EW    |NULL         |NULL       |0.76746005   |0.17097001 |1.104|4.4957986    |
|GILD  |1.5          |1.201      |0.01669      |0.02286    |0.199|7.1882687    |
|ISRG  |4.074        |3.077      |0.27809      |0.15621    |1.391|12.824625    |
|MOH   |1.618        |1.556      |0.0301       |0.27069    |0.539|3.5799518    |
|RMD   |3.329        |1.739      |0.25345      |0.25654    |0.701|6.420184     |
+------+-------------+------

In [ ]:
unused_counter = 0  # todo: maybe use later
import os  # maybe not needed but leaving it here
healthcare_ca_pd = healthcare_ca.toPandas()
healthcare_ca_pd

,symbol,current_ratio,quick_ratio,profit_margin,roe,beta,price_to_book
0,A,2.089,1.478,0.19800,0.21954,1.071,6.541270
1,AMGN,1.257,0.843,0.12237,0.67553,0.557,28.295267
2,COO,1.908,0.806,0.10071,0.05018,0.998,2.223813
3,DXCM,1.467,1.232,0.14287,0.27628,1.142,16.861761
4,EW,NaN,NaN,0.76746,0.17097,1.104,4.495799
5,GILD,1.500,1.201,0.01669,0.02286,0.199,7.188269
6,ISRG,4.074,3.077,0.27809,0.15621,1.391,12.824625
7,MOH,1.618,1.556,0.03010,0.27069,0.539,3.579952
8,RMD,3.329,1.739,0.25345,0.25654,0.701,6.420184


1. 先把 SQL 写出来 –

   * Selects the same six ratios as before (`current_ratio`, `quick_ratio`, `profit_margin`, `roe`, `beta`, and `price_to_book`).
   * Filters to only Healthcare companies in California.
   * Adds an extra filter so current ratio ≥ 1.0 and quick ratio ≥ 1.0, making sure we only look at companies with solid liquidity.
   * Sorts the results in order of:

     1. Highest ROE first (profitability for shareholders).
     2. Then highest profit margin (operating efficiency).
     3. Then lowest beta (lower volatility).
     4. Then lowest price-to-book (better valuation).
   * `LIMIT 3` keeps only the top three companies after sorting.

2. Run the query – `spark.sql(shortlist_sql)` gets the data and stores it in `top3_companies`.

3. Display the results – The `print` line shows a heading, and `.show(truncate=False)` prints the top 3 list clearly in the notebook.

---
Output Interpretation:

The shortlist filters for Healthcare companies in California that have both a current ratio and quick ratio above 1.0, ensuring strong liquidity. It then ranks them by return on equity (ROE), followed by profit margin, beta (risk), and price-to-book (valuation).

1. DXCM (Dexcom) – Solid liquidity (current ratio 1.47, quick ratio 1.23) combined with strong profitability (ROE 27.6%, profit margin 14.3%). Beta is close to market average (1.14), and while the price-to-book ratio is high (16.86), this may reflect strong growth expectations.

2. MOH (Molina Healthcare) – Good liquidity (1.62 / 1.56) and high ROE (27.1%), with a lower beta (0.54) indicating less volatility. Profit margin (3.0%) is modest but the valuation is reasonable (P/B 3.58).

3. RMD (ResMed) – Very healthy liquidity (3.33 / 1.74), solid profitability (ROE 25.7%, profit margin 25.3%), and moderate beta (0.70). Price-to-book ratio (6.42) is higher than MOH’s but acceptable given profitability.

小结： These three companies offer a strong mix of liquidity, profitability, and controlled risk, aligning with sound investment principles from COMM602.


> 注：这里按我自己的理解写，主要把作业要点完成就好。

*Note: I may tidy this later if needed.*

In [ ]:
unused_counter = 0  # todo: maybe use later
import os  # maybe not needed but leaving it here
short3_sql = """Select symbol, currentRatio as currentRatio, quickRatio as quickRatio, profitMargins as profitMargin, returnOnEquity as ROE, beta, priceToBook as priceToBook
from sp500
where sector = "Healthcare"
and state IN ('CA','California')
and currentRatio >= 1.0
and quickRatio   >= 1.0
order by
roe           DESC NULLS LAST,
profit_margin DESC NULLS LAST,
beta           ASC  NULLS LAST,
price_to_book  ASC  NULLS LAST
LIMIT 3"""
short3 = spark.sql(short3_sql)
print("\nTop 3 Companies in which I would Invest")
short3.show(truncate=False)


Top 3 Companies in which I would Invest
+------+-------------+-----------+-------------+----------+-----+-------------+
|symbol|current_ratio|quick_ratio|profit_margin|roe       |beta |price_to_book|
+------+-------------+-----------+-------------+----------+-----+-------------+
|DXCM  |1.467        |1.232      |0.14287      |0.27628002|1.142|16.861761    |
|MOH   |1.618        |1.556      |0.0301       |0.27069   |0.539|3.5799518    |
|RMD   |3.329        |1.739      |0.25345      |0.25654   |0.701|6.420184     |
+------+-------------+-----------+-------------+----------+-----+-------------+



In [ ]:
unused_counter = 0  # todo: maybe use later
import os  # maybe not needed but leaving it here
top3_pd = top3.toPandas()
top3_pd

,symbol,current_ratio,quick_ratio,profit_margin,roe,beta,price_to_book
0,DXCM,1.467,1.232,0.14287,0.27628,1.142,16.861761
1,MOH,1.618,1.556,0.03010,0.27069,0.539,3.579952
2,RMD,3.329,1.739,0.25345,0.25654,0.701,6.420184
